In [6]:
# 将图像切分保存下来

import numpy as np 
import cv2 

# h = 480, 
# w = 640
imgall = cv2.imread("assets/exposures.jpg")

IM_H = 240
IM_W = 320

img1 = imgall[0:IM_H, 0:IM_W]
img2 = imgall[0:IM_H, IM_W::]
img3 = imgall[IM_H::, 0:IM_W]
img4 = imgall[IM_H::, IM_W::]

cv2.imshow("src", imgall)
cv2.imshow("img1", img1)
cv2.imshow("img2", img2)
cv2.imshow("img3", img3)
cv2.imshow("img4", img4)

cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("img0.jpg", img1)
cv2.imwrite("img1.jpg", img2)
cv2.imwrite("img2.jpg", img3)
cv2.imwrite("img3.jpg", img4)

True

In [8]:
import numpy as np 
import cv2 

# 1. Loading exposure images into a list
img_fn = ["assets/img0.jpg", "assets/img1.jpg", "assets/img2.jpg", "assets/img3.jpg"]
img_list = [cv2.imread(fn) for fn in img_fn]

# 曝光时间
exposure_times = np.array([15.0, 2.5, 0.25, 0.0333], dtype=np.float32)


# 2. Merge exposures into HDR image
merge_debevec = cv2.createMergeDebevec()
hdr_debevec = merge_debevec.process(img_list, times=exposure_times.copy())
merge_robertson = cv2.createMergeRobertson()
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy())

# 3. Tonemap HDR image
tonemap1 = cv2.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())
res_robertson = tonemap1.process(hdr_robertson.copy())

# 4. Merge exposures using Mertens fusion
merge_mertens = cv2.createMergeMertens()
res_mertens = merge_mertens.process(img_list)

# 5. Convert to 8-bit and save
res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')
res_robertson_8bit = np.clip(res_robertson*255, 0, 255).astype('uint8')
res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')

cv2.imshow("ldr_debevec.jpg", res_debevec_8bit)
cv2.imshow("ldr_robertson.jpg", res_robertson_8bit)
cv2.imshow("fusion_mertens.jpg", res_mertens_8bit)

cv2.waitKey(0)
cv2.destroyAllWindows()
